In [4]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [39]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("final_dataset.csv")
print(data_initial.shape)

(3569, 694)


In [40]:
data_no_na = data_initial.dropna(axis = 1) # data should already have no NAs due to numerical imputation
print(data_no_na.shape)
i = 0
#for col in data_no_na.columns:
#    print("" + str(i) + ": " + col)
#    i = i + 1

include_col_indices = [0, 48]  # Start with individual indices

# Add ranges using loops
include_col_indices.extend(range(63, 139))
include_col_indices.extend(range(159, 279))
include_col_indices.extend(range(299, 343))
include_col_indices.extend(range(623, 691))

print(include_col_indices)
print(len(include_col_indices))

# Subset columns from the NumPy array
subset_data = data_no_na.iloc[:, include_col_indices]

# Convert the subsetted array to a DataFrame
df_subset = pd.DataFrame(subset_data)

# Select columns of number data types
data = df_subset.select_dtypes(include=['number'])

y = data['FA_zone_time']
x = data.loc[:, data.columns != 'FA_zone_time']
x = x.loc[:, x.columns != 'FA_zone_time']
X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.6, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, train_size = 0.5, test_size=0.5, random_state=45)

# Above two lines in unison accomplish a 60-20-20 split for train-validation-test

(3569, 691)
[0, 48, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 299, 300, 301, 302, 303, 304, 305, 30

In [41]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 1000
val_batch_size = 1000
test_batch_size = 1000

cpu
cpu


In [42]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1:].values, dtype=torch.float32)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [43]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, 1000)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.batch_norm_1000 = nn.BatchNorm1d(1000)
        self.batch_norm_500 = nn.BatchNorm1d(500)
        self.batch_norm_400 = nn.BatchNorm1d(400)
        self.batch_norm_300 = nn.BatchNorm1d(300)
        self.batch_norm_200 = nn.BatchNorm1d(200)
        self.batch_norm_100 = nn.BatchNorm1d(100)

        self.fc2 = nn.Linear(1000, 1000)

        self.fc3 = nn.Linear(1000, 1000)

        self.fc4 = nn.Linear(1000, 500)

        self.fc5 = nn.Linear(500, 500)

        self.fc6 = nn.Linear(500, 400)

        self.fc7 = nn.Linear(400, 300)

        self.fc8 = nn.Linear(300, 200)

        self.fc9 = nn.Linear(200, 100)

        self.fc10 = nn.Linear(100, 1)

    def forward(self, x):
        #print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_500(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_500(x)

        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_400(x)

        x = self.fc7(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_300(x)

        x = self.fc8(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_200(x)

        x = self.fc9(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_100(x)

        x = self.fc10(x)
        
        return x


In [44]:
num_epochs = 50
trial_count = 30

learning_rate = 0.01 # in last optimization, best lr ended up being 0.0016
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.view(-1, 1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += train_batch_size
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        #model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for (X, y) in valid_loader:
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.view(-1, 1))
                val_loss += loss.float().item()
                count += val_batch_size
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 300, 600)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    print("learning rate:", lr)
    print("weight_decay:", weight_decay)
    print("hidden_size:", hidden_size)
    print("dropout_rate:", dropout_rate)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

input_dim = 309
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)

In [45]:
hidden_size = 500
dropout_rate = 0.25
weight_decay = 0.000001
learning_rate = 0.01

# Above inspired by last run best params, which were:
# {'lr': 0.0016359428701992723, 'weight_decay': 3.4162832861696797e-06, 'hidden_size': 493, 'dropout_rate': 0.4524812840729844}
model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()


# # Train the model with the best hyperparameters on the entire dataset
# for epoch in range(num_epochs):
#     ########################### Training #####################################
#     model.train(True)
#     train_loss = 0
#     count = 0
#     for batch, (X, y) in enumerate(train_loader):
#         X = X.float().to(device)
#         y = y.float().to(device)
#         prediction = model(X).to(device)
#         optimizer.zero_grad()
#         loss = loss_func(prediction, y.view(-1, 1).float().to(device))
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.float().item()
#         count += 1
#     scheduler.step()
#     print("Epoch", epoch+1, "Training loss:", train_loss / count)

# # Save the best model
# torch.save(model.state_dict(), "faceoffs_off_off_state_dict2.pt")
# # save the trained model
# torch.save(model, "faceoffs_off_off2.pt")

Epoch 1 Training loss: 731.6875813802084
Epoch 2 Training loss: 630.9138590494791
Epoch 3 Training loss: 743.2991536458334
Epoch 4 Training loss: 680.441162109375
Epoch 5 Training loss: 620.7219441731771
Epoch 6 Training loss: 658.5355021158854
Epoch 7 Training loss: 548.1417439778646
Epoch 8 Training loss: 535.0118001302084
Epoch 9 Training loss: 550.2408650716146
Epoch 10 Training loss: 502.1156514485677
Epoch 11 Training loss: 528.8517557779948
Epoch 12 Training loss: 501.62054443359375
Epoch 13 Training loss: 505.22580973307294
Epoch 14 Training loss: 498.5453796386719
Epoch 15 Training loss: 481.10064697265625
Epoch 16 Training loss: 441.4634704589844
Epoch 17 Training loss: 469.9709981282552
Epoch 18 Training loss: 468.3215637207031
Epoch 19 Training loss: 510.55030314127606
Epoch 20 Training loss: 434.3927001953125


In [46]:
# Optuna Hyperparameter Tuning
import optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

C:\Users\Tad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-08-29 20:41:02,483] A new study created in memory with name: no-name-3875ebc9-65eb-4b04-a830-52e88ed037b1


learning rate: 0.0016201586298246254
weight_decay: 4.3394834205798547e-05
hidden_size: 458
dropout_rate: 0.2331684137024599

EPOCH 1 of 20

Training loss: 0.6598216756184896
Validation loss: 0.6832701416015625

EPOCH 2 of 20

Training loss: 0.6471678059895833
Validation loss: 4.25683740234375

EPOCH 3 of 20

Training loss: 0.6635911661783854
Validation loss: 32.357580078125

EPOCH 4 of 20

Training loss: 0.6965250651041667
Validation loss: 1.6983831787109376

EPOCH 5 of 20

Training loss: 0.684968505859375
Validation loss: 0.546871826171875

EPOCH 6 of 20

Training loss: 0.6980617879231771
Validation loss: 1.6001920166015624

EPOCH 7 of 20

Training loss: 0.6926809895833334
Validation loss: 0.5414559936523438

EPOCH 8 of 20

Training loss: 0.6412587280273437
Validation loss: 0.5402754516601562

EPOCH 9 of 20

Training loss: 0.6226188863118489
Validation loss: 0.5386800537109375

EPOCH 10 of 20

Training loss: 0.7013902180989583
Validation loss: 0.5781532592773437

EPOCH 11 of 20

Train

[I 2023-08-29 20:42:47,108] Trial 0 finished with value: 0.5496725463867187 and parameters: {'lr': 0.0016201586298246254, 'weight_decay': 4.3394834205798547e-05, 'hidden_size': 458, 'dropout_rate': 0.2331684137024599}. Best is trial 0 with value: 0.5496725463867187.


Validation loss: 0.5496725463867187
{'lr': 0.0016201586298246254, 'weight_decay': 4.3394834205798547e-05, 'hidden_size': 458, 'dropout_rate': 0.2331684137024599}


In [51]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

num_epochs = 5 # training for more epochs on best model (which will be saved)

train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

# Train the model with the best hyperparameters on the entire dataset
# for epoch in range(num_epochs):
#     ########################### Training #####################################
#     model.train(True)
#     train_loss = 0
#     count = 0
#     for batch, (X, y) in enumerate(train_loader):
#         X = X.float().to(device)
#         y = y.float().to(device)
#         prediction = model(X).to(device)
#         optimizer.zero_grad()
#         loss = loss_func(prediction, y.view(-1, 1).float().to(device))
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.float().item()
#         count += 1
#     print("Epoch", epoch+1, "Training loss:", train_loss / count)
#     scheduler.step()
#     model.eval()
#     val_loss = 0
#     count = 0
#     with torch.no_grad():
#         for (X, y) in valid_loader:
#             X = X.to(device)
#             y = y.to(device)
#             prediction = model(X).to(device)
#             loss = loss_func(prediction, y.view(-1, 1))
#             val_loss += loss.float().item()
#             count += val_batch_size
#         print("Validation loss:", val_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoff_mod_state_dict.pt")
# save the trained model
torch.save(model, "faceoff_mod.pt")

{'lr': 0.0016201586298246254, 'weight_decay': 4.3394834205798547e-05, 'hidden_size': 458, 'dropout_rate': 0.2331684137024599}

EPOCH 1 of 5

Training loss: 0.6297891845703125
Validation loss: 0.7173461303710937

EPOCH 2 of 5

Training loss: 0.6995457763671875
Validation loss: 6.4935859375

EPOCH 3 of 5

Training loss: 0.5985120646158855
Validation loss: 172.67725

EPOCH 4 of 5

Training loss: 0.7675731811523437
Validation loss: 0.553473876953125

EPOCH 5 of 5

Training loss: 0.7130353800455729
Validation loss: 1.2157974853515625


In [52]:
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print("Loss:", test_loss)
print(f"R-squared Coefficient: {r2:.4f}")


Test Loss: 1319.6432
True labels: [11.0, 41.0, 28.0, 1.0, 8.0, 10.0, 21.0, 9.0, 11.0, 1.0, 23.0, 15.0, 11.0, 6.0, 7.0, 8.0, 5.0, 14.0, 1.0, 6.0, 1.0, 2.0, 4.0, 7.0, 8.0, 7.0, 1.0, 1.0, 3.0, 6.0, 22.0, 21.0, 48.0, 7.0, 8.0, 13.0, 27.0, 78.0, 11.0, 9.0, 21.0, 10.0, 9.0, 28.0, 94.0, 3.0, 6.0, 14.0, 8.0, 13.0, 20.0, 7.0, 6.0, 1.0, 9.0, 16.0, 12.0, 14.0, 12.0, 1.0, 19.0, 48.0, 4.0, 3.0, 4.0, 13.0, 8.0, 43.0, 16.0, 5.0, 59.0, 6.0, 15.0, 19.0, 24.0, 1.0, 22.0, 13.0, 5.0, 16.0, 1.0, 10.0, 14.0, 6.0, 8.0, 44.0, 4.0, 16.0, 3.0, 1.0, 8.0, 45.0, 4.0, 9.0, 9.0, 1.0, 6.0, 14.0, 48.0, 17.0, 48.0, 18.0, 3.0, 7.0, 3.0, 33.0, 10.0, 5.0, 37.0, 4.0, 5.0, 6.0, 5.0, 15.0, 60.0, 10.0, 18.0, 1.0, 5.0, 2.0, 10.0, 49.0, 9.0, 18.0, 8.0, 27.0, 82.0, 18.0, 10.0, 2.0, 28.0, 3.0, 7.0, 4.0, 8.0, 10.0, 16.0, 16.0, 80.0, 14.0, 12.0, 25.0, 4.0, 145.0, 9.0, 8.0, 30.0, 1.0, 77.0, 4.0, 106.0, 9.0, 3.0, 2.0, 25.0, 11.0, 5.0, 20.0, 9.0, 7.0, 1.0, 69.0, 8.0, 122.0, 1.0, 26.0, 12.0, 4.0, 21.0, 20.0, 6.0, 29.0, 3.0, 44.0, 6.0, 